In [1]:

import pandas as pd
import csv
import streamlit as st
import pandas as pd
import requests
from typing import List
from utils import load_and_preprocess_data

In [32]:
pd.set_option('display.max_columns', 500)

def parse_survey_answers_files():
    df = pd.read_csv('./data/survey.csv')
    df.dropna()
    df = df.drop(
        ['W skali od 1 do 10 jak bardzo lubisz słone jedzenie', 'W skali od 1 do 10 jak bardzo lubisz słodkie jedzenie',
         'W skali od 1 do 10 jak bardzo lubisz gorzkie jedzenie', 'W skali od 1 do 10 jak bardzo lubisz mięso',
         'W skali od 1 do 10 jak bardzo lubisz pikantne jedzenie',
         'W skali od 1 do 10 jak bardzo lubisz kuchnie śródziemnomorską',
         'W skali od 1 do 10 jak bardzo lubisz kuchnie azjatycką',
         'W skali od 1 do 10 jak bardzo lubisz kuchnie amerykańską',
         'W skali od 1 do 10 jak bardzo lubisz kuchnie arabską',
         'W skali od 1 do 10 jak bardzo lubisz owoce morza(również ryby)',
         'W skali od 1 do 10 jak bardzo lubisz tłuste jedzenie', 'W skali od 1 do 10 jak bardzo lubisz kuchnie Polską'
         ],
        axis=1)
    columns = df.columns
    df.T.insert(0, "questions", columns)
    return df

parse_survey_answers_files().head()


,Adres e-mail,Avocado Chicken Parmigiana,Easy Weeknight Beef Stroganoff,Celery Apple Risotto With Crispy Pancetta,Blackberry Creme Brulee,Sheet Pan Butternut Squash with Spicy Italian Sausage,Brussels Sprout Carbonara with Fettuccini,Irish Soda Bread By Mommie Cooks,Strawberry Cheesecake Chocolate Crepes,"Ravioli With Arugula, Pine Nuts, Raisins & Cream",Alaskan Smoked Salmon Nicoise Salad With Alouette Crumbled Feta,Creamy Alfredo and Shrimp Pasta,Gnocchi and Wilted Spinach Salad,Make Olive Garden’s Delicious Pasta Fagioli At Home,German Lemon Cake With Cranberry Aka Cranberry Zitronenkuchen,"Sweet Potato, Sausage and Caramelized Onion Pizza",Greek Chicken Sheet Pan Dinner with Green Beans and Feta,Hungarian Potato Soup,Savory Carrot Souffle,Roasted Tomato and Garlic Quiche,Spinach Goats Cheese Roulade Main Dish,Southwest Blt Crostini,Corn Flan Side Dish,BRUSCHETTA STUFFED PORTOBELLOS,Rosemary Rum Raisin Soda Bread with Pecans,Honey Panna Cotta With Blueberries and Graham Crackers,Beef Wellington,Lemon-Pepper Fettucine Alfredo,Savory Radicchio and Prosciutto Crostini Topped with Sweet Syrupy Sapa,Open Ravioli With Butternut Squash and Amaretti,Spanish Tortilla,"Easy Chicken, Kielbasa and Shrimp Paella",Mini Meatball & Ravioli Soup With Garlic Croutons,Golden Beet Panzanella,Light and Easy Alfredo,Gyros,Herbed Vegetables In A Feta Quiche With A Wild Rice Crust,Eggplant Fries with Tzatziki Sauce,Corned Beef And Cabbage With Irish Mustard Sauce,Berry and White Chocolate Trifle,Slow Cooker Shanghai-Style Beef Borscht,Roasted Fish Bruschetta,Niçoise Salad,Clams With Spanish Sausage,Sun-Dried Tomato Romesco,X-Country Double Lobster Risotto,Jen's Swedish Meatballs,Spanish style salmon fillets,"Butternut, Chicken and Feta Risotto",Linguine Alla Carbonara,Leek and Mushroom Quiche,Classic scones,Fava Crostini,Frikadellen German Meat Patties,Individual Tiramisu Parfaits,Polenta gnocchi with savoy cabbage and cheese,Italian Seafood Stew,Rich Jelly Scones,Dried Fruit and Ginger Scones,How to Make the Ultimate Christmas Fruit Pizza,https://spoonacular.com/recipeImages/1061943-312x231.jpg,Chicken Farfalle with Low-Fat Alfredo Sauce,Simple Broccoli Cheddar Quiche,Bbq Chicken and Goat Cheese Ravioli,Nutmeg and Rosemary Butternut Squash Risotto,What to make for dinner tonight?? Bruschetta Style Pork & Pasta,Crispy Italian Cauliflower Poppers Appetizer,Italian Pasta Salad with organic Arugula,Broccolini Quinoa Pilaf,Irish Whiskey Pie,Lemon and Edamame Risotto with Pecans,Fresh Cherry Scones,Italian Style Meatballs,Apple Cinnamon Quiche,Beet Gnocchi With Steak and Brown Butter Sauce,Peanut Butter Cup Trifle,Italian Tuna Pasta,Savory Cheese Dill Scones,Greek Yogurt Chicken Salad,Dried Fruit and Ginger Scones.1,Gluten And Dairy Free Alfredo Lasagna,Kale Colcannon,Fallen Chocolate Cakes With Spanish Paprika,Beef Stroganoff w Bella Mushrooms,Salmon and Broccoli Crepes,Romanian Pea and Chicken Stew,German Rhubarb Cake with Meringue,Pernik - Czech Spice Cake,Black Forest Cake For Blogger Friends,Ratatouille Pasta,Panna Cotta with Raspberry and Orange Sauce,Asparagus Quiche,Pumpkin Banana Protein Oat Bars,Strawberries and Cream Scones,Fresh Tomato Risotto with Grilled Green Vegetables,Zucchini Quiche & Almond Flax Crust,Minute Tiramisu,Brown Butter and Sage Ravioli,Chicken Cacciatore,Italian String Beans With Anchovies and Breadcrumbs,Italian Beef Braciole,Colcannon,Mushroom and Parmesan Brown Rice Risotto,Greek Inspired Spring Omelet,Greek Shrimp Orzo,Coconut Thai Risotto With Seared Scallops,Kale Bruschetta,Spanish Meatballs In Tomato Sauce,Mussels in Irish Ale,"fresh corn, roasted tomato & pickled garlic pizza with cornmeal crust",Mushroom & Gruyere Quiche,Corned Beef and Cabbage,Chicken/sweet Potato Roulade With Goat Cheese Sauce,German Meatloaf Falscher Hase,Borscht Soup,"Focaccia with Tomato, Green Onion and Garlic",Cheese Ravioli,Butternut Squash Pizza,Heart Shaped Spinach Ravioli,Roasted Pumpkin and Barley Risotto,"Roasted Cranberries, Ricotta 

In [2]:
def load_and_preprocess_data():
    df = pd.read_csv(
        "data/ratings.csv",
        encoding="utf-8",
    )
    # Remove nans values
    df = df.dropna()

    # Get unique entries in the dataset of users and products
    users = df["Adres e-mail"].unique()

    products = df["pytanie"].unique()
    df['ocena'] = df['ocena'].astype(int)
    # Create a categorical type for users and product. User ordered to ensure
    # reproducibility
    user_cat = pd.CategoricalDtype(categories=sorted(users), ordered=True)
    product_cat = pd.CategoricalDtype(categories=sorted(products), ordered=True)
    # Transform and get the indexes of the columns
    user_idx = df["Adres e-mail"].astype(user_cat).cat.codes
    product_idx = df["pytanie"].astype(product_cat).cat.codes

    # Add the categorical index to the starting dataframe
    df["Adres e-mail"] = user_idx
    df["pytanie"] = product_idx

    return df, user_idx, product_idx, user_cat.categories, product_cat.categories


In [3]:
from typing import Dict, Any, Union
from recommender import Recommender


In [12]:
data, email_order, dishes_order, sorted_users, sorted_dishes = load_and_preprocess_data()
recommender = Recommender(
        data['ocena'],
        email_order,
        dishes_order,
    )




In [13]:
recommender.create_and_fit(
    "als",
        # Fine-tuned values
        model_params=dict(
            factors=190,
            alpha=0.6,
            regularization=0.06,
            random_state=42,
        ),
    # model_params=dict(
    #         factors=10,
    #         alpha=0.3,
    #         regularization=0.02,
    #         random_state=56,
    #     ),
)

F:\uczelnia\AI_DIET_GENERATOR\rec_system\venv\lib\site-packages\implicit\utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

In [19]:
suggestions_and_score  = recommender.recommend_products(
                0, 30
            )
suggestions_and_score

(array([182, 128,  99,  23, 166, 143, 218,  44, 125,  56,  17,  53, 216,
         87, 195, 206, 149,  22,  45,  86,   3,  19,   1, 174,  49, 119,
        210, 135, 122,  38]),
 array([0.00280243, 0.0006966 , 0.00068401, 0.00054979, 0.00047768,
        0.00039763, 0.00038963, 0.00037594, 0.0003744 , 0.00035101,
        0.00034731, 0.00033666, 0.00033216, 0.00033108, 0.00031941,
        0.00031791, 0.00031584, 0.00031176, 0.00029786, 0.00028915,
        0.00028484, 0.00026762, 0.00025985, 0.00025872, 0.00025656,
        0.00025562, 0.00025467, 0.00025178, 0.00023302, 0.00022903],
       dtype=float32))

In [21]:
for sas in suggestions_and_score[0]:
    print(f"meal -> {sorted_dishes[sas]}")
recommender.model.similar_users(0)

meal -> Sauteed Italian Eggplant Side Dish
meal -> Light Greek Lemon Chicken Orzo Soup
meal -> Individual Tiramisu Parfaits
meal -> Brown Butter and Sage Ravioli
meal -> Ratatouille Pasta
meal -> Mushroom and Parmesan Brown Rice Risotto
meal -> fresh corn, roasted tomato & pickled garlic pizza with cornmeal crust
meal -> Coconut Thai Risotto With Seared Scallops
meal -> Lemon Pasta Alfredo (Vegan)
meal -> Crock Pot Shredded French Dip
meal -> Berry and White Chocolate Trifle
meal -> Creamy Risotto With Smoked Turkey Sausage
meal -> Zucchini Quiche & Almond Flax Crust
meal -> Greek Shrimp Orzo
meal -> Spanish Meatballs In Tomato Sauce
meal -> Vegan Colcannon Soup
meal -> Panna Cotta with Raspberry and Orange Sauce
meal -> Broccolini Quinoa Pilaf
meal -> Colcannon
meal -> Greek Inspired Spring Omelet
meal -> Asparagus Lemon Risotto
meal -> Black Forest Cake For Blogger Friends
meal -> Almost Heaven Cake
meal -> Roasted Tomato Crostini with Olive Tapenade
meal -> Corned Beef And Cabbage W

(array([ 0,  2,  8, 47, 35,  5, 13, 17, 40, 44]),
 array([1.        , 0.9999683 , 0.9998975 , 0.99989367, 0.9998259 ,
        0.9998204 , 0.19511843, 0.19511001, 0.19506508, 0.19505195],
       dtype=float32))

In [67]:


recommender.explain_recommendation(
            45, suggestions_and_score[0][1], 5
        )

# n_recommended = len(suggestions_and_score)
# for suggestion in suggestions_and_score:
#     explained = recommender.explain_recommendation(
#         45, suggestion, n_recommended
#     )
    # suggested_items_id = [id[0] for id in explained]
    # print(suggested_items_id)

[(61, 0.1658995172565235),
 (52, 0.0820034115743634),
 (14, 0.06406558164493147),
 (203, 0.056311534532062425),
 (196, 0.048221179695381804)]

In [85]:
df = pd.read_csv(
        "./data/ratings.csv",
        encoding="utf-8",
    )
df['pytanie'].sort_values().unique()

array(['Alaskan Smoked Salmon Nicoise Salad With Alouette Crumbled Feta',
       'Almost Heaven Cake', 'Apple Cinnamon Quiche',
       'Asparagus Lemon Risotto', 'Asparagus Quiche', 'Austrian Goulash',
       'Avocado Chicken Parmigiana', 'BRUSCHETTA STUFFED PORTOBELLOS',
       'Baked Chicken Caprese', 'Baked Ravioli & Meat Sauce',
       'Baked Swedish Pancake', 'Baked Ziti Casserole',
       'Bbq Chicken and Goat Cheese Ravioli',
       'Beef Stroganoff w Bella Mushrooms', 'Beef Wellington',
       'Beet Gnocchi With Steak and Brown Butter Sauce', 'Berry Trifle',
       'Berry and White Chocolate Trifle', 'Best Chicken Parmesan',
       'Black Forest Cake For Blogger Friends', 'Blackberry Creme Brulee',
       'Borscht Soup', 'Broccolini Quinoa Pilaf',
       'Brown Butter and Sage Ravioli',
       'Brussels Sprout Carbonara with Fettuccini', 'Buckwheat Crepes',
       'Buffalo Mozzarella & Pepperoni Pizza with a Cauliflower Crust',
       'Butternut Squash Pizza', 'Butternut, Chick

In [27]:
from data.survey import update_survey_answers_file
update_survey_answers_file()


In [26]:
from data.survey import convert_to_implicit_dataset
convert_to_implicit_dataset().to_csv('./data/ratings.csv')

In [31]:
from data.recipes import get_all_recipes
test = get_all_recipes('f1a65dd7832647d18888c171f92ed1b7')

In [34]:
test

,id,title
0,715769,Broccolini Quinoa Pilaf
1,715495,Turkey Tomato Cheese Pizza
2,715573,Simple Skillet Lasagna
3,659109,Salmon Quinoa Risotto
4,648279,Italian Tuna Pasta
5,648257,Italian Steamed Artichokes
6,660266,Slow Cooked Corned Beef and Cabbage
7,648247,Italian Seafood Stew
8,640819,Crispy Italian Cauliflower Poppers Appetizer
9,1095806,Spanish style salmon fillets
